[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/text-to-video-synthesis-colab/blob/main/potat1_video_webui_colab.ipynb)

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
# !pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 -U

!git clone -b v2.3 https://github.com/camenduru/ui
!git clone https://github.com/camenduru/tunnels /content/ui/extensions/tunnels
!git clone https://github.com/camenduru/t2v /content/ui/extensions/t2v

%cd /content/ui

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d /content/ui/models/Stable-diffusion -o v1-5-pruned-emaonly.ckpt

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/potat1-with-text-encoder-original-format/resolve/main/VQGAN_autoencoder.pth -d /content/ui/models/ModelScope/t2v -o VQGAN_autoencoder.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/potat1-with-text-encoder-original-format/resolve/main/open_clip_pytorch_model.bin -d /content/ui/models/ModelScope/t2v -o open_clip_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/potat1-with-text-encoder-original-format/resolve/main/text2video_pytorch_model.pth -d /content/ui/models/ModelScope/t2v -o text2video_pytorch_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/potat1-with-text-encoder-original-format/raw/main/configuration.json -d /content/ui/models/ModelScope/t2v -o configuration.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ModelScope/resolve/main/model.ckpt -d /content/ui/models/VideoCrafter -o model.ckpt

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/ui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/ui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/ui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple --disable-safe-unpickle